# This Building Does Not Exist (Yet)

## AIASF _NEXT_ 2019
### Tyler Kvochick
### TEECOM Research & Development

# Goals

## Beyond Metaphors

What is a neural network, literally?

## Inner Workings

What makes a neural network...work?

## Applications

What interesting models exist today and what can we make them do?

# Setup

* Download a premade module for convenience
* Install a network visualization module
* Import libraries

In [0]:
!pip install torchviz

In [0]:
import sys
import os
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torchviz import make_dot
import torchvision.transforms as transforms
import torchvision.utils as vutils
import matplotlib.pyplot as plt

# Beyond Metaphors

What does a neural network look like?

In [0]:
# Network visualization class
class NetworkVisualization(nn.Module):
    def __init__(self):
        super(NetworkVisualization, self).__init__()
        
        self.conv0 = nn.Conv2d(3, 6, (3, 3))
        self.act0 = nn.LeakyReLU()
        self.conv1 = nn.Conv2d(3, 6, (3, 3))
        self.act1 = nn.LeakyReLU()
    
    def forward(self, x):
        x0 = self.act0(self.conv0(x))
        x1 = self.act1(self.conv1(x))
        
        return x0 + x1

In [0]:
# Use our new class
nv = NetworkVisualization().cuda()

fake_image = torch.randn(1, 3, 512, 512).cuda()

out = nv(fake_image)

nv

In [0]:
# Render the graph
make_dot(out.mean())

In [0]:
# Render the parameters
params = [p for p in list(nv.conv0.parameters())]
params

# No More Metaphors (Summary)

## Multiple Representations of NNs

1. As executable code / structured data (software)
2. As a connective graph (visualization)
3. As lists of numbers organized into rectilinear shapes (tensors)

---

# Inner Workings

We have seen that deep learning models are full of orthogonal collections of numbers that we call tensors. The rest of the model is made of mathematical functions for combining those tensors.

## Convolution

For working with images, the most common function for combining tensors in a learnable way is called convolution.

Convolution is a confusing name for "searching for known patterns".

To get a better intuitive understanding of what convolution is, we are going to look at a 1D example.

In [0]:
lim = 8 * np.pi
x_axis = np.linspace(-lim, lim, 200)
sin_x = np.sin(x_axis)

plt.plot(x_axis, sin_x)

In [0]:
# Put special_signal in an external file

def special_signal():
    low = -12 * np.pi
    high = 4 * np.pi
    
    space_s = np.linspace(2 * low, 2 * high, 210)
    space = np.linspace(-8 * np.pi, 8 * np.pi, 210)
    
    noise = np.random.randn(space.size) * 0.03
    
    event = np.sin(space_s) / space_s
    
    base_signal = np.sin(space) * noise
    
    return base_signal + event
    
ss = special_signal()

plt.plot(ss)
plt.vlines([140, 171], -0, 1, linestyles="--")

In [0]:
label = np.zeros_like(ss)
label[140:171] = 1.0

plt.plot(label)

In [0]:
a = np.arange(0, 10)
print(a)
print(a[2:5])

In [0]:
# Define convolution from scratch

def convolve(signal, kernel, stride=1):
    sw = signal.shape[0]
    kw = kernel.shape[0]
    
    n_positions = sw // stride
    step_size = sw // n_positions

    out = []
    
    input_slices = []
    
    for i in range(n_positions):
        start_idx = i * step_size
        
        this_slice = signal[start_idx:start_idx + kw]
        
        tsw = this_slice.shape[0]
        
        # Ensure that our output is the same shape as our kernel
        this_slice = np.pad(this_slice, (0, kw - tsw), 'constant')
        
        input_slices += [this_slice]

        product = this_slice * kernel
        
        out += [np.sum(product)]

    return np.array(out), np.stack(input_slices)
        

In [0]:
# Examine one step of applying convolution

kernel = np.random.randn(30)

output, input_slices = convolve(ss, kernel, stride=1)

plt.plot(output)
plt.plot(ss)
plt.plot(label)

In [0]:
# Define a loss function
def squared_error(output, target):
    error = target - output
    sq_error = (error ** 3) / np.abs(error)
    return sq_error

se = squared_error(output, label)
plt.plot(output)
plt.plot(ss)
plt.plot(label)
plt.plot(se)

## Things Required to Make Machines Learn

1. Input
2. Label

(Dataset)

3. Function to transform Input -> Label
4. Function to measure how far off the output is

(Model)

5. A way to update kernel based on how far off the output is

(Backpropagation, i.e. derivatives)

$$ O(\mathbf{I}) = \mathbf{I} \cdot \mathbf{k} $$
$$\frac{\delta O}{\delta \mathbf{k}} = \mathbf{I} $$


In [0]:
# Apply our convolution function to learn about our event

kernel = np.random.randn(30)
learning_rate = 0.0005
losses = []

for n in range(1000):
    output, input_slices = convolve(ss, kernel)
    
    se = squared_error(output, label)
    
    loss = se.mean()
    
    losses += [loss]
    
    if n % 10 == 0:
        print("Mean Squared Error Loss: {:.4f}".format(loss))
    
    # Manual backpropagation
    for (error, in_slice) in zip(se, input_slices):
        kernel_update = error * in_slice * learning_rate
        kernel = kernel + kernel_update
    
losses = np.array(losses)       

In [0]:
# Plot output and label

plt.plot(output)
plt.plot(label)

In [0]:
# Smooth and plot

def gaussian_kernel(mu=0, sigma=1, width=30):
    x = np.linspace(-sigma, 2 * sigma, width)
    exp = np.e ** (-1 * (x - mu) ** 2)/(2 * sigma ** 2)
    
    return (1 / (sigma * np.sqrt(2 * np.pi))) * exp

smooth, _ = convolve(output, gaussian_kernel())
plt.plot(smooth / smooth.max())
plt.plot(label)

In [0]:
# Plot and print kernel

plt.plot(kernel)
kernel

In [0]:
# Plot loss over time

plt.plot(losses)

# How does it work (Summary)

Neural networks use (fairly) simple mathematical functions to discover non-obvious solutions.
These are not found by describing complex rules, but merely by describing relationships between inputs and desired outputs.

We can use ideas from calculus and linear algebra to start from random values that transform from input to output and steer the output towards better and better results.

# Applications

## Generative Models

One of the fascinating things about deep neural nets is that they can learn arbitrary, _qualitative_ functions.

Functions like "generate images that are similar to this collection of images".

And this can be done with operations similar to what we have just written.

In [0]:
# Trasposed Convolution

x = torch.randn(1, 1, 8)
print(x.shape)
x

In [0]:
tc = nn.ConvTranspose1d(1, 1, 4, 2, 1, bias=False)
tc

In [0]:
y = tc(x)
print(y.shape)
y

In [0]:
y = tc(y)
print(y.shape)
y

In [0]:
make_dot(y)

In [0]:
# Download module
!curl https://raw.githubusercontent.com/TEECOM/this-building-does-not-exist/spaceheater-training/ml/python/DrawinGAN.py --output DrawinGAN.py

In [0]:
# Download dataset

!wget https://www.dropbox.com/sh/v7uu10rkve2vnt8/AAA6InT1OjUquORG0i1syD7ka?dl=0 -O ALotOfPlans.zip
!unzip ALotOfPlans.zip -d .

In [0]:
!ls ALotOfPlansModified/

In [0]:
!wget https://www.dropbox.com/s/2y9vp2u0c4mautt/1561057242-discriminator.pth?dl=0 -O discriminator.pth
!wget https://www.dropbox.com/s/67n1i8ffqdsaedy/1561057242-generator.pth?dl=0 -O generator.pth

In [0]:
from DrawinGAN import Generator, Discriminator, Container, Trainer

In [0]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, latent_dim=512):
        super(ConvBlock, self).__init__()

        self.latent_dim = latent_dim

        self.a = Generator.A(in_channels, latent_dim=latent_dim)

        self.ada_in = Generator.AdaIN()

        self.conv = nn.ConvTranspose2d(
            in_channels,
            out_channels,
            4,
            2,
            1,
            bias=False
            )

        self.bn = nn.BatchNorm2d(out_channels)
        self.act = nn.LeakyReLU()

    def forward(self, container):
        nx, cx, hx, wx = container.x.shape

        y = self.a(container.w)

        x = self.ada_in(container.x, y)

        x = self.conv(container.x)

        x = self.bn(x)

        x = self.act(x)

        return Container(x, container.w)

In [0]:
generator = Generator.DrawingGenerator(ConvBlock, output_channels=1).cuda()
discriminator = Discriminator.DrawingDiscriminator(input_channels=1).cuda()

In [0]:
device = torch.device("cuda")

generator.load_state_dict(torch.load("generator.pth", map_location="cuda:0"))
discriminator.load_state_dict(torch.load("discriminator.pth", map_location="cuda:0"))

In [0]:
dataloader, n_batches = Trainer.image_dataset("alotofplansmodified", batch_size=4)

In [0]:
import matplotlib
%matplotlib inline
Trainer.notebook_train(generator, discriminator, dataloader, n_batches, plt)

In [0]:
to_image = transforms.ToPILImage()

z = torch.randn(4, 512, 1, 1).cuda()

fake_images = generator.forward(z, None, mode="generator")

image_grid = vutils.make_grid(fake_images.clone().cpu(), nrow=2, normalize=True)
image_grid= to_image(image_grid)

image_grid
    